# An Introduction to MAST

## Table of Contents
- [Introduction](#introduction)
- [Overview](#overview)
- [Search Tools](#search-tools)
    - [MAST Portal](#MAST-Portal)
    - [MAST Search](#MAST-Search)
    - [exo.MAST](#exoMAST)
    - [Virtual Observatory](#virtual-observatory)
- [MAST Tools](#MAST-tools)
    - [astroquery.mast](#astroquery.mast)
    - [APIs](#APIs)
    - [High Level Science Products](#HLSPs)
    - [TIKE: Cloud Science](#TIKE)

## Introduction

Astronomy is one of the most open sciences when it comes to access to data repositories, and exoplanetary data is no exception. NASA and other space agencies release the data that they've been collecting for decades, leading to some truly massive data sets available online. It's a large universe - it makes sense that the repositories would be similarly large. While there are a handful of common repositories, each features a unique way to access and handle these large sources of data. 

This lesson will give an overview on MAST, the tools it provides, some activities to learn how to access the data, and some responsible practices when it comes to larger scale analysis.

## Overview

MAST (the Mikulski Archive for Space Telescopes) is a NASA-funded data archive, focused on observations in the near-infrared, optical, and ultraviolet. MAST contains data from missions such as the Hubble Telescope, JWST (James Webb Space Telescope), TESS (Transiting Exoplanet Survey Satellite), and the Kepler Space Telescope. MAST is also not just a host for data, but provides a number of resources to easily access and work with their data.

You can access MAST __[here](https://archive.stsci.edu/)__ to explore the site.


## Search Tools

Essential to navigating the huge amounts of data available is MAST's numerous search tools. They all function slightly differently, so be sure to learn the strengths of each one to understand what suits your research needs best.

### MAST Portal

The MAST Portal is MAST's basic and most comprehensive search engine. You can find access to the web app __[here](https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html)__. The site provides access to a large number of NASA missions, catalogs, and other repositories around the world.

### MAST Search



### exo.MAST

__[exo.MAST](https://exo.mast.stsci.edu/)__ is MAST's dedicated exoplanetary data search engine, and the most powerful tool for exoplanet analysis. It accesses data sets from Kepler, K2, Hubble, TESS and JWST. These typically include light curves and spectroscopic data. This section will focus on the online portal, but information regarding the exo.MAST API can be found under the API section. 

We can search by looking for an individual object of interest or by searching through exo.MAST's table of atmospheric data. A dropdown menu appears while typing, so if looking for planets around a certain star,  they appear in a list. 

![alt text](<Screenshot 2024-06-12 140413.png>)

After selecting a certain planet or object, you'll be redirected to a webpage with further information and data. 

Similar to NASA's NEA, exo.MAST provides basic known data on the left side of the page, including measurements of the star, planet, and system as a whole. On the top, there'll be a dropdown menu listing the mission where this data was sourced from, as well as alternative data sources from alternative missions. 

![alt text](<Screenshot 2024-06-12 164033.png>)

However, the value of the site comes from the data visible on the right side. You'll see tabs on the top right that control what you see. Typically, you'll start on "Data Coverage", where you can click and drag an area to select all data sets relevant for that period, and then scroll down to download the datasets themselves, and view relevant information. There can be multiple seperate observations associated with the same point in the planet's period, representative of repeated observation.

The "Light Curve" tabs are calibrated light curves from KEPLER or TESS data, and you can explore/interact with the graphs or download the data yourself. It is important to understand that this is not "raw" data. The details of calibration include a number of strategies to process raw data for analysis, including **folding**, **binning**, and **detrending**. 

> **Folding** is the process of stacking several observations of a periodic event (such as an exoplanet transit) and averaging across observations to procure a noiseless observation. This removes random noise that changes over observations, and allows for a analysis of a single light curve. 

> **Binning** is the process of "binning" together subsequent data points. For instance, all data points within 5 seconds periods could be averaged together. This produces a smoother curve and reduces small-scale noise to show large-scale trends in data, at the cost of time resolution.

> **Detrending** is the process of removing trends from a dataset to only show the difference in the data from a certain trend. This removes systematic biases and allows researchers to focus on smaller fluctuations within the data. 

![alt text](<Screenshot 2024-06-12 164316.png>)

The "Related Links" tab allows you to explore any relevant papers already published about your object, or dive into further analyses of the data measured.

Finally, the "JWST Visibility" allows users to plan for a JWST observation. It provides relevant information for an observing session, such as when the target is visible to JWST, signal-to-noise ratios for atmospheric spectroscopy, and links to other helpful sources.

#### Exoplanet Atmosphere Observability Table

Found __[here](https://catalogs.mast.stsci.edu/eaot)__. [To finish]

### Virtual Observatory

### TESSCut

## MAST Tools

MAST offers plenty of tools, both developed by NASA and community-derived, to ease processing and analysis. These range from custom preselected packages of exoplanet data to python packages to cloud processing, so you don't need to download huge datasets to analyze them.

### astroquery.MAST

astroquery.MAST allows for easy querying of the MAST Portal through python. While you can find the detailed documentation __[here](https://astroquery.readthedocs.io/en/latest/mast/mast.html)__, this notebook will go over the basics of how to make requests. 

The package allows for 3 main kinds of queries: observational data, catalog data, and image data. These are provided by classes that can be imported from the package, such as `Observations` and `Catalogs`. Let's see an example of how to use the `Observations` class for exoplanetary data.

In [4]:
import astroquery.mast as aqm

The `Observations` class input is similar to the MAST Portal's filtering. The simplest queries can be made with by position or by target name, and they output an `astropy` table of observations that qualify. Let's look at some examples.

In [12]:
query_table = aqm.Observations.query_object('HAT-P-7b')
print(len(query_table))
# Let's see just how large of a table... 
# "len()" outputs the length (or number of rows) 

8840


Woah, that's a lot of observations - that's what we get for not filtering! While there are some uses for a search this broad, let's look at some applications for adding some parameters. 

You can see some parameters already, as the column labels when we listed the table out. "intentType" provides either "science" or "calibration" - you'll rarely run into calibration datasets (they make up 0.5% of all data on MAST), but it's good to filter them out nonetheless. "obs_collection" provides the mission that particular dataset is from, such as TESS, HST (Hubble), JWST, etc. You can find a full list of all the columns and their descriptions __[here](https://mast.stsci.edu/api/v0/_c_a_o_mfields.html)__.

The `query_object` only allows you to add a "radius of search" parameter object, so we'll be using the `query_criteria` method. 

Let's look for lightcurves of HAT-P-7b taken by the TESS telescope. This allows us to narrow down our search significantly!

In [10]:
query_table = aqm.Observations.query_criteria(objectname="HAT-P-7b", 
                                              obs_collection="TESS",
                                              dataproduct_type=["timeseries"])
# indenting like this doesn't affect the code output, but is just a way to try and make it more readable/organized

print(len(query_table))
print(query_table[:10])

30
intentType obs_collection provenance_name ...   objID1       distance     
---------- -------------- --------------- ... --------- ------------------
   science           TESS            SPOC ...  70448544 287.98135262165573
   science           TESS            SPOC ...  70449515  638.6413483917396
   science           TESS            SPOC ...  70449529                0.0
   science           TESS            SPOC ...  70526761 287.98135262165573
   science           TESS            SPOC ...  70531322                0.0
   science           TESS            SPOC ...  70544780  638.6413483917396
   science           TESS            SPOC ... 116683518                0.0
   science           TESS            SPOC ... 116846050                0.0
   science           TESS            SPOC ... 117053213                0.0
   science           TESS            SPOC ... 117053215 118.92799122073585


Alright, much better - now how do we actually use this data? The most straightforward way is to straight up download it, which is exactly what `download_products` does. 

[TO COMPLETE - HOW TO DOWNLOAD PRODUCTS WITH JUPYTER NOTEBOOK? STORED WITHIN NOTEBOOK?]

In [15]:
data_products = aqm.Observations.get_product_list('70449529')
print(len(data_products))
downloads = aqm.Observations.download_products(data_products)
print(downloads)

1
                                                             Local Path                                                              ...
------------------------------------------------------------------------------------------------------------------------------------ ...
./mastDownload\HLSP\hlsp_qlp_tess_ffi_s0035-0000000033221567_tess_v01_llc\hlsp_qlp_tess_ffi_s0035-0000000033221567_tess_v01_llc.fits ...


To mention: Using Cloud-based programming to get around downloading enourmous amounts of data - what other strategies?

### APIs

#### MAST API

#### exo.MAST API

Documentation __[here](https://exo.mast.stsci.edu/docs/)__.

### High Level Science Products

### TIKE: Cloud Science